In [6]:
import itertools
import numpy as np
starts = np.arange(0, 9001, 200)
ends = np.arange(200, 9201, 200)
ends[-1] = 9045
expts = ['baseline_bal', 'reduction', 'adult']
data_index = range(ends.shape[0])
iteration = range(10)
lrs = [1e-2, 1e-3, 4e-4, 1e-4]
a = itertools.product(expts, data_index, iteration, lrs)

In [2]:
b = [i for i in a]

In [3]:
len(b)

5520

In [10]:
5520/3 


1840.0

In [2]:
from scipy.stats import norm
norm.cdf(0)

0.5

In [9]:
enumerate(b)

In [2]:
import itertools
expts = ['reduction', 'baseline_bal', 'sensr'] 
iteration = range(10)
lrs = [1e-2, 1e-3, 4e-4, 1e-4]

a = itertools.product(expts, iteration, lrs)
b = [i for i in a]

In [3]:
len(b)

120